In [1]:
import poppler

In [154]:
doc = poppler.load_from_file("/data/Basic Rules (2014).pdf")

In [ ]:
doc.infos()

In [ ]:
page = doc.create_page(7)
page.text()

In [ ]:
boxes = page.text_list(page.TextListOption.text_list_include_font)
for b in boxes:
    print(b.text, b.get_font_name(), b.get_font_size())

In [ ]:
import pandas as pd

data = [
    (p, i, b.text, b.get_font_name(), b.get_font_size())
      for p in range(doc.pages)
      for i, b in enumerate(doc.create_page(p).text_list(page.TextListOption.text_list_include_font))
]
df = pd.DataFrame(data, columns=["page", "word_in_page", "text", "font", "size"])
df

In [ ]:
df.query("size > 20 and size < 30")

In [ ]:
chapter_headers = df.query("size > 30 and size < 35")
section_headers = df.query("size > 20 and size < 30")
section_headers

In [ ]:
df.query("size > 15 and size < 20").groupby("page")

In [ ]:
chapter_headers.groupby("page")

In [167]:
section_starts = section_headers.groupby("page")[["page", "word_in_page"]].transform(lambda x: min(x)).drop_duplicates()

In [ ]:
chapter_headers.groupby("page")["text"].transform(lambda x: " ".join(x)).drop_duplicates()

In [ ]:
for pg, fonts in doc.create_font_iterator():
    print(pg, [f.name for f in fonts])

In [ ]:
chapter_starts = chapter_headers.groupby("page")[["page", "word_in_page"]].transform(lambda x: min(x)).drop_duplicates()
chapter_starts

In [ ]:
chapter_heading = []
section_heading = []
current_doc = []

in_chapter_heading = False
in_section_heading = False
last_font_size = 0

docs = {}
def stringify(thing):
    return " ".join(thing)

for p in range(doc.pages):
    page = doc.create_page(p)
    for b in page.text_list(page.TextListOption.text_list_include_font):
        font_size = b.get_font_size()

        if font_size < last_font_size:
            current_doc.append("\n--------\n")

        if font_size < 20:
            if font_size > last_font_size:
                current_doc.append("\n")
            current_doc.append(b.text)

        if font_size > last_font_size:
            if font_size > 20:
                # end of section
                docs[f"{stringify(chapter_heading)}{(' - ' + stringify(section_heading)) if section_heading else ''}"] = f"{stringify(chapter_heading)}\n\n{stringify(section_heading)}\n{stringify(current_doc)}"
                current_doc = []
                section_heading = []
                if font_size > 30:
                    # end of chapter
                    chapter_heading = []

        if font_size > 30 and font_size < 35:
            chapter_heading.append(b.text)

        if font_size > 20 and font_size < 30:
            section_heading.append(b.text)

        last_font_size = font_size
docs

In [ ]:
from utils.elastic import elastic_request
import requests
test = elastic_request(method=requests.post, url="_ingest/pipeline/clean_and_embed/_simulate", data={"docs":[{"_source": {"content": docs["Preface"]}}]})
test.json()

In [ ]:
docs_to_ingest = []

for cursor in range(len(chapter_starts)):
    start = chapter_starts.iloc[cursor]
    print(df.query(f"page == {start.page} and word_in_page >= {start.word_in_page}")["text"].values[0])
    qry = f"page >= {start.page} and word_in_page > {start.word_in_page}"
    if cursor+1 < len(chapter_starts):
        end = chapter_starts.iloc[cursor+1]
        qry += f" and page <= {end.word_in_page} and word_in_page < {end.word_in_page}"

    sections_in_chapter = section_starts.query(qry)
    if sections_in_chapter.empty:
        docs_to_ingest.append(df.query(qry)["text"].transform(lambda x: " ".join(x)).values)
    for cursor2 in range(len(sections_in_chapter)):
        pass   


# each section (defined from start word of a header to start word of the next header)
#for cursor in range(len(section_starts)):
#    start = section_starts.iloc[cursor]
    #print(df.query(f"page == {start.page} and word_in_page == {start.word_in_page}")["text"].values[0])
#    if cursor+1 < len(section_starts):
#        end = section_starts.iloc[cursor+1]
    #    print(cursor, start.page, start.word_in_page, '-->', end.page, end.word_in_page)
    #else:
    #    print(cursor, start.page, start.word_in_page, '-->', "END")
    
    # get contents of section
    # prepend header for chapter the section is in
    
# for each section header in the chapter (we know because it appears on a page between chapter N and chapter N+1)
# create a doc containing everything between the section header and the next section header
docs_to_ingest

In [ ]:
df

In [ ]:
"finer" in docs["Preface"]